In [4]:
import sys
sys.path.append("../")

In [5]:
from typing import Tuple, Dict
from collections import defaultdict
from ds4400_final_project.dataset.common import train_and_evaluate_classifier, print_classifier_results
from sklearn.svm import SVC

In [6]:
# list of all kernels to try
kernels = ["linear", "poly", "rbf", "sigmoid"]
# list of all datasets to try
csv_files = ["features_3_sec.csv", "features_30_sec.csv"]

# { CSV_FILENAME: { SOLVER: ( TRAIN_ACC, TEST_ACC, FAILED_COUNT ) }
results: Dict[str, Dict[str, Tuple]] = defaultdict(dict)

# iterate over all solvers
for kernel in kernels:
	# iterate over all datasets
	for filename in csv_files:

		# construct the model
		classifier = SVC(kernel=kernel)

		# train and evaluate the model
		train_acc, test_acc, failed_count = train_and_evaluate_classifier(filename, classifier)
		results[filename][kernel] = (train_acc, test_acc, failed_count)

In [7]:
for filename, result in results.items():
	for kernel, (train_acc, test_acc, failed_count) in result.items():
		print_classifier_results(f"{filename} ({kernel})", train_acc, test_acc, failed_count)

features_3_sec.csv (linear)
Train accuracy: 81.025
Train error:    18.975
Test accuracy:  76.281
Test error:     23.719

Genre           | # of Incorrect | % of All Incorrect
----------------------------------------------------
[01] ROCK       | 418            | 20.4%
[02] REGGAE     | 310            | 15.1%
[03] COUNTRY    | 272            | 13.3%
[04] DISCO      | 267            | 13.0%
[05] HIPHOP     | 241            | 11.7%
[06] BLUES      | 153            | 7.5%
[07] POP        | 136            | 6.6%
[08] JAZZ       | 121            | 5.9%
[09] METAL      | 101            | 4.9%
[10] CLASSICAL  | 33             | 1.6%


features_3_sec.csv (poly)
Train accuracy: 87.091
Train error:    12.909
Test accuracy:  79.921
Test error:     20.079

Genre           | # of Incorrect | % of All Incorrect
----------------------------------------------------
[01] ROCK       | 269            | 17.6%
[02] HIPHOP     | 221            | 14.5%
[03] BLUES      | 218            | 14.3%
[04] REGGAE     